# Playing with PREM

This notebook is in two parts, the first shows the code used to make some of the graphs
used in my slides. The  second part shows how we can make a version of PREM that can be
altered. Your task is to see how much you can alter the density distribution in the Earth
without altering the total mass or moment of inertia by too much. More information is
below.

In [ ]:
# Start by importing some modules
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
%matplotlib inline

# The code for this module can be found in the
# earth_model.py file. It contains code for PREM's
# somewhat odd peicewise polynomials, and knows how
# to integrate them. Only pressure calculation is done
# numerically.
import earth_model

## Create a default `earth_model` and  see what  it looks like

The next few cells show how properties derived from PREM's description ofEarth's density can be
plotted and  evaluated.

In [ ]:
# The default model is "normal" PREM (as defined in the paper)
prem = earth_model.Prem()

In [ ]:
# Make a graph of the density distribution
fig, ax = plt.subplots(figsize=(10,6))

rs = np.arange(0, 6371, 0.5)
ax.plot(rs, prem.density(rs), 'k')

ax.set_xlabel('Radius (km)')
ax.set_ylabel('Density (kg/m$^3$)')

ax.axvline(1221.5, ls=':', c='k')
ax.axvline(3480, ls='--', c='k')
ax.axvline(3630, ls=':', c='k')
ax.axvline(5701, ls=':', c='k')
ax.axvline(5971, ls=':', c='k')

secax = ax.secondary_xaxis('top', functions=(lambda x: 6371 - x, lambda x: 6371 - x))
secax.set_xlabel('Depth (km)')


plt.show()

### Mass and moment of inertia

The total mass of the Earth is found by integrating the density:

$$ M = 4\pi \int_0^{R_{e}} \rho(r) r^2 \,\mathrm{d}r.$$

For the Earth $M$ is $5.9722\times10^{24}$ kg with an uncertainty of $6\times10^{20}$ kg. This is slightly 
less than the best estimate from the 1970s used in the construction of PREM.

The moment of inertia of the Earth (around the rotation axis) is also given by integrating the density: 

$$I = \frac{2}{3} 4\pi \int_0^{R_{e}} \rho(r) r^4 \,\mathrm{d}r.$$

However, we typically think of the moment of inertia factor, $\frac{I}{MR_{e}^2}$ for planets as this is
then comparable for different sized bodies. Earth has a moment of inertia factor of 0.3307. A homogeneous
has a moment of inertia factor of 0.4 which is reduced as mass is concentrated into the center. 

In [ ]:
print("Mass of the Earth is:", prem.mass(prem.r_earth), "kg")
print("Earth's moment of inertia is: ", prem.moment_or_inertia(prem.r_earth)[0], "kg m^2")
print("I/MR**2:", prem.moment_or_inertia(prem.r_earth)[1])

# What does it look like?
fig, ax = plt.subplots(figsize=(10,6))

rs = np.arange(0, 6371, 0.5)
ax.plot(rs, prem.mass(rs)/1e24, 'g')

ax.set_xlabel('Radius (km)')
ax.set_ylabel('Mass inside radius ($10^{24}$ kg)')

ax.axvline(1221.5, ls=':', c='k')
ax.axvline(3480, ls='--', c='k')
ax.axvline(3630, ls=':', c='k')
ax.axvline(5701, ls=':', c='k')
ax.axvline(5971, ls=':', c='k')

secax = ax.secondary_xaxis('top', functions=(lambda x: 6371 - x, lambda x: 6371 - x))
secax.set_xlabel('Depth (km)')

plt.show()

### Gravity and pressure

From the distribution of mass we can calculate the acceleration due to gravity for each radial shell: 

$$g(r) = \frac{G M(r)}{r^2} $$

and from this we can evaluate the pressure:

$$P(r) = \int_{R_e}^r -g(r) \rho(r) \,\mathrm{d}r $$

In [ ]:
print("Surface gravity:", prem.gravity(6371), "m/s^2")
print("Pressure at center of Earth:", prem.pressure(0.0), "GPa")
print("Pressure at CMB:", prem.pressure(3480.0), "GPa")

# What does it look like?
fig, ax = plt.subplots(figsize=(10,6))

rs = np.arange(0, 6371, 0.5)
ax.plot(rs, prem.gravity(rs), 'k')

ax.set_xlabel('Radius (km)')
ax.set_ylabel('Gravity (m/s$^2$)')

ax2 = ax.twinx() 
ax2.plot(rs, prem.pressure(rs), 'g')
ax2.set_ylabel('Pressure (GPa)', color='g')
ax2.tick_params(axis='y', labelcolor='g')


ax.axvline(1221.5, ls=':', c='k')
ax.axvline(3480, ls='--', c='k')
ax.axvline(3630, ls=':', c='k')
ax.axvline(5701, ls=':', c='k')
ax.axvline(5971, ls=':', c='k')

secax = ax.secondary_xaxis('top', functions=(lambda x: 6371 - x, lambda x: 6371 - x))
secax.set_xlabel('Depth (km)')

plt.show()

## Play with PREM's density parameters

The density model is defined by a set of 13 polynomials stitched together at first and second order discontinuities. That is, the density is given by: 

$$
\rho(r) = \left\{
\begin{array}{ll}
      \rho_{0,0} + \rho_{0,1}r + \rho_{0,2}r^2 + \rho_{0,3}r^3 & r\leq 1221.5 \; \mathrm{km} \\
      \rho_{1,0} + \rho_{1,1}r + \rho_{1,2}r^2 + \rho_{1,3}r^3 & 1221.5\leq r\leq 3480.0 \; \mathrm{km}\\
      \vdots & \vdots \\
      \rho_{12,0} + \rho_{12,1}r + \rho_{12,2}r^2 + \rho_{12,3}r^3 & 6368.0\leq r\leq 6371.0 \; \mathrm{km} \\
\end{array} 
\right.
$$

The `earth_model` module knows how to integrate this kind of function. The next cell shows how to set up a
PREM-like model which can be modified by changing values directly. Play with this model to see how much you
can reduce the mass of the core (and increase the mass of the mantle) without changing gross properties of the
Earth. Can you add a low(er) density layer at the top of the outer core? 

In [ ]:
# This implements the PREM density model with a density gradient in the inner core
# You really only need to worry about the density_params and breakpoints

density_params = np.array([[13.0885,  -1.5, -8.8381,  0.0000], # in PREM second parameter is 0.0
                           [12.5815, -1.2638, -3.6426, -5.5281],
                           [7.9565, -6.4761,  5.5283, -3.0807],
                           [7.9565, -6.4761,  5.5283, -3.0807],
                           [7.9565, -6.4761,  5.5283, -3.0807],
                           [5.3197, -1.4836,  0.0000,  0.0000],
                           [11.2494, -8.0298,  0.0000,  0.0000],
                           [7.1089, -3.8045,  0.00002,  0.0000],
                           [2.6910,  0.6924,  0.0000,  0.0000],
                           [2.6910,  0.6924,  0.0000,  0.0000],
                           [2.9000,  0.0000,  0.0000,  0.0000],
                           [2.6000,  0.0000,  0.0000,  0.0000],
                           [1.0200,  0.0000,  0.0000,  0.0000]])


# Turn range of polynomials from 0 - 1 to 0 - r_earth (makes mass easer)
# and put density into kg/m^3. This basically converts from the parameters
# given in the PREM paper to SI units for density and km for radius.
r_earth = earth_model._r_earth
density_params[:,0] = density_params[:,0] * 1000
density_params[:,1] = (density_params[:,1] * 1000) / r_earth 
density_params[:,2] = (density_params[:,2] * 1000) / (r_earth**2)
density_params[:,3] = (density_params[:,3] * 1000) / (r_earth**3)


# All 14 discontiuities in PREM in km.
breakpoints = np.array([0.0, 1221.5, 3480.0, 3630.0, 5600.0, 5701.0, 5771.0,
                        5971.0, 6151.0, 6291.0, 6346.6, 6356.0, 6368.0, 6371.0])



my_prem = earth_model.Prem(breakpoints=breakpoints, density_params=density_params)

In [ ]:
# Make a graph showing the two density profiles and report key values
fig, ax = plt.subplots(figsize=(10,6))

rs = np.arange(0, 6371, 0.5)
ax.plot(rs, prem.density(rs), 'k')
ax.plot(rs, my_prem.density(rs), 'r--')

ax.set_xlabel('Radius (km)')
ax.set_ylabel('Density (kg/m$^3$)')

ax.axvline(1221.5, ls=':', c='k')
ax.axvline(3480, ls='--', c='k')
ax.axvline(3630, ls=':', c='k')
ax.axvline(5701, ls=':', c='k')
ax.axvline(5971, ls=':', c='k')

secax = ax.secondary_xaxis('top', functions=(lambda x: 6371 - x, lambda x: 6371 - x))
secax.set_xlabel('Depth (km)')


plt.show()

print("Mass of the Earth (PREM) is:", prem.mass(prem.r_earth), "kg")
print("Mass of the Earth (my_PREM) is:", my_prem.mass(my_prem.r_earth), "kg")

print("I/MR**2 (PREM) is:", prem.moment_or_inertia(prem.r_earth)[1])
print("I/MR**2 (my_PREM) is:", my_prem.moment_or_inertia(my_prem.r_earth)[1])